# Getting Started with APIs for Data Engineers

This notebook provides a detailed walkthrough of how to interact with APIs using Python's `requests` library. We'll cover the following topics:
    1. Introduction to APIs
    2. Authentication
    3. Parameters
    4. Making API Requests
    5. Parsing API Responses
    6. Error Handling
    7. Working with Multiple APIs

In [27]:
import requests
import pandas as pd
pd.set_option('display.max_colwidth', None)

## 1. Introduction to APIs

An API (Application Programming Interface) is a set of rules that allows different software applications to communicate with each other. APIs often provide access to data and services that can be used in your applications. For data engineers, APIs are a critical tool for integrating external data sources into your data pipelines.

### Example: What is an API? 

## 2. Authentication

Authentication is the process of verifying your identity when interacting with an API. Most APIs require an API key or token to access their data.

### Example 1: Authentication with OpenWeatherMap API

In the OpenWeatherMap API, authentication is done by passing an API key as a parameter in the URL.

In [ ]:
response = requests.get("http://api.openweathermap.org/data/2.5/weather?q=London,uk&appid=your_openweathermap_api_key")
print(response.text)

**Note:** Here, the `appid` parameter is used for authentication.

### Example 2: Authentication with Football Data API
The Football Data API requires the API key to be passed in the request headers.

In [ ]:
response = requests.get("https://v3.football.api-sports.io/leagues", headers={
    'x-rapidapi-key': 'your_football_api_key',
    'x-rapidapi-host': 'v3.football.api-sports.io'
})
print(response.text)

**Note:** The API key is included in the headers as `x-rapidapi-key`.

## 3. Parameters

Parameters in API requests allow you to filter, sort, and customize the data returned by the API. They are often passed in the URL as query parameters.

### Example 1: Query Parameters for Filtering

In this example, we use the `q` parameter to specify the city (London, UK) and the `appid` parameter for authentication.


In [ ]:
response = requests.get("http://api.openweathermap.org/data/2.5/weather?q=London,uk&appid=your_openweathermap_api_key")
print(response.text)

**Note:** The `q` parameter filters the data to return only the weather for London, UK.

### Example 2: Geographical Parameters

Parameters like `lat` (latitude) and `lon` (longitude) allow you to request data for a specific geographical location.

### Example 3: Making Requests with the Football API

The [Football API](https://dashboard.api-football.com/) is a bit more complex and has more detailed and vast information than the weather API. Lets use the [league](https://v3.football.api-sports.io/leagues). If we make a request to the endpoint without parameters, it returns information for all the leagues in the world and the records, as you can imagine that is a whole lot of information, so we will streamline the output using parameters to get information for the [UEFA Championship League](https://www.uefa.com/uefachampionsleague/) for the 2024 season.

In [25]:
url = "https://v3.football.api-sports.io/leagues"

payload={'id':2,
        'season':2024}#passing in parameters to filter for only the UEFA champion league 2024
headers = {
  'x-rapidapi-key': SAPI,
  'x-rapidapi-host': 'v3.football.api-sports.io'
}

response = requests.request("GET", url, headers=headers, params=payload)
uefa_league=response.json()

In [28]:
df=pd.DataFrame.from_dict(uefa_league['response'])
df

,league,country,seasons
0,"{'id': 2, 'name': 'UEFA Champions League', 'type': 'Cup', 'logo': 'https://media.api-sports.io/football/leagues/2.png'}","{'name': 'World', 'code': None, 'flag': None}","[{'year': 2024, 'start': '2024-07-09', 'end': '2025-01-29', 'current': True, 'coverage': {'fixtures': {'events': True, 'lineups': True, 'statistics_fixtures': True, 'statistics_players': True}, 'standings': True, 'players': True, 'top_scorers': True, 'top_assists': True, 'top_cards': True, 'injuries': True, 'predictions': True, 'odds': True}}]"


The response is deeply nested and can most definitely benefit from further parsing. We can also use the fixtures endpoint to get match fixtures for the teams and other information like player statistics. You can experiment with these further.

In [ ]:
response = requests.get("http://api.openweathermap.org/data/2.5/weather?lat=6.4550575&lon=3.3941795&appid=your_openweathermap_api_key")
print(response.text)

## Section 2: Using Variables for API Keys and Parameters

So far, we’ve been hardcoding the API key and other parameters directly into the URL. this is not good practice, Subsequently we will be passing in our API keys as environment variables, this is standard practice, we will be using the loadenv library, althought there are other libraries too for managing environment variables

In [9]:
from dotenv import load_dotenv
import os
WAPI = os.getenv("WAPI")#loading the weather API
SAPI = os.getenv("SAPI")#loading the sports API

In [5]:
load_dotenv()

True

In [26]:
print("\n# Example 4: Using variables for API key and parameters")
city = "Paris"
url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={WAPI}&units=metric"
response = requests.get(url)
data = response.json()
print(f"Temperature in {city}: {data['main']['temp']}°C")


# Example 4: Using variables for API key and parameters
Temperature in Paris: 19.75°C


In [39]:
params = {
 "q": 'Lagos',
 "units": "metric",
 "appid": WAPI # API key as a query parameter
 }
url = f"https://api.openweathermap.org/data/2.5/weather"#?q={city}&appid={WAPI}&units={units}"
response = requests.get(url, params=params)
data = response.json()
print(f"Temperature in {data['name']}: {data['main']['temp']}°C")

Temperature in Lagos: 29.18°C


## 5. Parsing API Responses

After making an API request, the response is typically returned in JSON format. Parsing this response allows you to extract the data you need.

### Example: Parsing a JSON Response


In [ ]:
response = requests.get("http://api.openweathermap.org/data/2.5/weather?q=London,uk&appid=your_openweathermap_api_key")
data = response.json()
print(f"City: {data['name']}, Temperature: {data['main']['temp']}K")

**Note:** Here, we're extracting the city name and temperature from the JSON response.

## 6. Error Handling

When interacting with APIs, it's important to handle errors that may occur, such as invalid requests or server issues.

### Example: Basic Error Handling


In [ ]:
response = requests.get("http://api.openweathermap.org/data/2.5/weather?q=InvalidCity&appid=your_openweathermap_api_key")
if response.status_code == 200:
    data = response.json()
    print(f"City: {data['name']}, Temperature: {data['main']['temp']}K")
else:
    print(f"Failed to retrieve data. Status code: {response.status_code}")
    print(f"Error message: {response.text}")

**Note:** This code checks if the request was successful (`status_code` 200). If not, it prints an error message.

## 7. Working with Multiple APIs

Sometimes, you'll need to work with multiple APIs in a single project. Here's an example of how you might combine data from the OpenWeatherMap API and the Football Data API.


## Conclusion

In this notebook, we've covered the basics of working with APIs using the `requests` library. We've explored how to authenticate requests, use parameters, parse JSON responses, handle errors, and combine data from multiple APIs. Understanding these concepts is essential for data engineers who need to integrate external data sources into their workflows.
